# Web scrping - Servidores UFRN

## Descrição
O trabalho consiste em implementar e executar Web Scraping no portal da transparência para obter a listagem de servidores da UFRN. Para isso tem-se a seguir os blocos de códigos usados para realizar a coleta. Primeiramente tem-se a as importações necessárias para realizar a coleta:

```python
import pandas as pd
import numpy as np
from requests import get
from bs4 import BeautifulSoup
from time import sleep, time
from random import randint
from warnings import warn
from IPython.core.display import clear_output
```
Em seguida, a impletação do web scraping:

```python
paginas = [str(i) for i in range(1,414)]

requests = 0
start_time = time()
elapsed_time = 0
nomes = []

for pagina in paginas:
    url_UFRN = 'http://www.portaldatransparencia.gov.br/servidores/'\
        'OrgaoExercicio-ListaServidores.asp?CodOrg=26243&Pagina={}'.format(pagina)
    response = get(url_UFRN)
    
    sleep(randint(1,6))
    
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    if response.status_code != 200:
       warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    if requests > 413:
        warn('Numero de requests maior que o espperado')  
        break
       
    pagina_soup = BeautifulSoup(response.text, 'html.parser')
    nomes_servidores = pagina_soup.select("div #listagem table tr td a")
    for nome_serv in nomes_servidores:
        nomes.append(nome_serv.text)
```

Ao final da coleta, a lista de servidores é salva em um CSV para processamento futuro:

```python
dfNomes = pd.DataFrame()
dfNomes['Nome do Servidor'] = nomes
dfNomes.to_csv('nomesServidores.csv')
```

Com o intuito de determinar o sexo de cada servidor através do nome uma função para extrair o primeiro nome foi criada:

```python
def extrairPrimeiroNome(nomes = [], *args):
    primeiroNome = []
    for nome in nomes:
        primeiroNome.append(nome.partition(' ')[0])
    return primeiroNome
```

Com o uso da função foi criado e armazenado uma lista de primeiros nomes para análise futura.
```python
dfPrimeiroNome = pd.DataFrame()
dfPrimeiroNome['Nome'] = extrairPrimeiroNome(nomes)
dfPrimeiroNome.to_csv('primeiroNomeServidores.csv')
```